In [1]:
!gdown --id 1OurDQUtbWQacvT32HMqFL7vIUrSMllOp


Downloading...
From: https://drive.google.com/uc?id=1OurDQUtbWQacvT32HMqFL7vIUrSMllOp
To: /content/preprocessed_data.csv
100% 300k/300k [00:00<00:00, 43.5MB/s]


In [2]:
#Importing the necessary packages
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
df=pd.read_csv('preprocessed_data.csv')#reading the file preprocessed_data.csv

In [4]:
df.head(4)#visulazing the DataFrame


,Unnamed: 0,source,target
0,0,"U wan me to ""chop"" seat 4 u nt?\n",Do you want me to reserve seat for you or not?\n
1,1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,3,I'm thai. what do u do?\n,I'm Thai. What do you do?\n


In [5]:
def preprocess(x):#for removingt the last character
  x=x[:-1]
  return x

In [6]:
df['source']=df['source'].apply(preprocess)
df['target']=df['target'].apply(preprocess)

In [7]:
df=df[['source','target']]
df.head()

,source,target
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [8]:
df.shape

(2000, 2)

In [9]:
df=df[df['source'].apply(len)<170]#removing sentences where source sentence is greater than 170
df=df[df['target'].apply(len)<200]#removing snetences where target sentence is greater than 200

In [10]:
df.shape#printing the shape

(1990, 2)

In [11]:
from sklearn.model_selection import train_test_split
X=df['source']
y=df['target']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.01)#splitting in the data in the ratio of 99:1
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(1970,)
(20,)
(1970,)
(20,)


Target:

In [12]:
target_tokenizer= Tokenizer(filters=None,char_level=True,lower=False)#tokenzing the target in character level
target_tokenizer.fit_on_texts(y_train)#fitting on the target train
target_vocab_size= len(target_tokenizer.word_index) + 1
print(len(target_tokenizer.word_index))#printing the vocabulary size

90


In [13]:
target_encoded_docs_train = target_tokenizer.texts_to_sequences(y_train)#converting target train into sequence
target_encoded_docs_test = target_tokenizer.texts_to_sequences(y_test)#converting target test into sequence

target_padded_docs_train = pad_sequences(target_encoded_docs_train,padding='post')#padding target train 
target_padded_docs_test = pad_sequences(target_encoded_docs_test,maxlen=target_padded_docs_train.shape[1],padding='post')#padding target test

Source:

In [14]:
source_tokenizer= Tokenizer(char_level=True,lower=False)#tokenzing the source in character level
source_tokenizer.fit_on_texts(X_train)#fitting on the source train
source_vocab_size= len(source_tokenizer.word_index) + 1
print(len(source_tokenizer.word_index))#printing the vocabulary size

103


In [15]:
source_encoded_docs_train = source_tokenizer.texts_to_sequences(X_train)#converting source train into sequence
source_encoded_docs_test = source_tokenizer.texts_to_sequences(X_test)#converting source train into sequence

source_padded_docs_train = pad_sequences(source_encoded_docs_train,maxlen=target_padded_docs_train.shape[1],padding='post')#padding source train 
source_padded_docs_test = pad_sequences(source_encoded_docs_test,maxlen=target_padded_docs_train.shape[1],padding='post')#padding source test 

In [16]:
#we are reshaping because sparse_categorical_entropy expects 3dimensions
target_padded_docs_train=target_padded_docs_train.reshape((*target_padded_docs_train.shape,1))
target_padded_docs_test=target_padded_docs_test.reshape((*target_padded_docs_test.shape,1))

In [17]:
print(target_padded_docs_train.shape)
print(target_padded_docs_test.shape)

(1970, 199, 1)
(20, 199, 1)


In [18]:
#we are reshaping because sparse_categorical_entropy expects 3dimensions
source_padded_docs_train=source_padded_docs_train.reshape((*source_padded_docs_train.shape,1))
source_padded_docs_test=source_padded_docs_test.reshape((*source_padded_docs_test.shape,1))

In [19]:
print(source_padded_docs_train.shape)
print(source_padded_docs_test.shape)

(1970, 199, 1)
(20, 199, 1)


In [20]:
X_train.to_csv('X_train2.csv')
y_train.to_csv('y_train2.csv')
X_test.to_csv('X_test2.csv')
y_test.to_csv('y_test2.csv')


In [21]:
import pandas as pd 
pd.DataFrame(source_encoded_docs_train).to_csv("source_encoded_docs_train2.csv")
pd.DataFrame(source_encoded_docs_test).to_csv("source_encoded_docs_test2.csv")
pd.DataFrame(target_encoded_docs_train).to_csv("target_encoded_docs_train2.csv")
pd.DataFrame(target_encoded_docs_test).to_csv("target_encoded_docs_test2.csv")


Model1:

In [22]:
input=tf.keras.layers.Input(shape=(199,))
embed=tf.keras.layers.Embedding(source_vocab_size,256, input_length=source_padded_docs_train.shape[1])(input)
lstm1=tf.keras.layers.LSTM(128, return_sequences=True)(embed)  
dense=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(512, activation='relu'))(lstm1)
drop=tf.keras.layers.Dropout(0.5)(dense)
output=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(target_vocab_size, activation='softmax'))(drop)
model=tf.keras.models.Model(inputs=input,outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 199)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 199, 256)          26624     
_________________________________________________________________
lstm (LSTM)                  (None, 199, 128)          197120    
_________________________________________________________________
time_distributed (TimeDistri (None, 199, 512)          66048     
_________________________________________________________________
dropout (Dropout)            (None, 199, 512)          0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 199, 91)           46683     
Total params: 336,475
Trainable params: 336,475
Non-trainable params: 0
_______________________________________________________

In [23]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [24]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=100,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/100
2/2 [==============================] - 9s 1s/step - loss: 4.1856 - accuracy: 0.3459 - val_loss: 2.7667 - val_accuracy: 0.6420
Epoch 2/100
2/2 [==============================] - 1s 520ms/step - loss: 2.1551 - accuracy: 0.6701 - val_loss: 1.4970 - val_accuracy: 0.6907
Epoch 3/100
2/2 [==============================] - 1s 532ms/step - loss: 1.4772 - accuracy: 0.6788 - val_loss: 1.3300 - val_accuracy: 0.6593
Epoch 4/100
2/2 [==============================] - 1s 545ms/step - loss: 1.3352 - accuracy: 0.6729 - val_loss: 1.3038 - val_accuracy: 0.6965
Epoch 5/100
2/2 [==============================] - 1s 501ms/step - loss: 1.3312 - accuracy: 0.6997 - val_loss: 1.3129 - val_accuracy: 0.6965
Epoch 6/100
2/2 [==============================] - 1s 518ms/step - loss: 1.3253 - accuracy: 0.6968 - val_loss: 1.3014 - val_accuracy: 0.6965
Epoch 7/100
2/2 [==============================] - 1s 515ms/step - loss: 1.3145 - accuracy: 0.6953 - val_loss: 1.2833 - val_accuracy: 0.6965
Epoch 8/100
2/2 

In [25]:
#https://machinelearningmastery.com/beam-search-decoder-natural-language-processing/
from math import log
from numpy import array
from numpy import argmax
import numpy as np
def beam_search_decoder(data, k):
	sequences = [[list(), 0.0]]
	# walk over each step in sequence
	#print(sequences)
	for row in data:
		all_candidates = list()
		# expand each current candidate
		for i in range(len(sequences)):
			seq, score = sequences[i]
			for j in range(len(row)):
				candidate = [seq + [j], score - np.log(row[j])]
				all_candidates.append(candidate)
		# order all candidates by score
		ordered = sorted(all_candidates, key=lambda tup:tup[1])
		sequences = ordered[:k]
	return sequences
 

In [29]:
def prediction(x):

  index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
  index_to_words[0] = '<PAD>'

  y=''.join([index_to_words[prediction] for prediction in x])
  return y
for i in range(20):
  print("Input text: ")
  a=list(X_test[i:i+1])
  print(a[0])

  print("Actual Output: ")
  b=list(y_test[i:i+1])
  print(b[0])

  print("Predicted Output for beam==3 : ")
  x=model.predict(source_padded_docs_test[i:i+1])

  res=beam_search_decoder(x[0],3)

  y1=prediction(res[0][0])
  y1=y1.split(' ')
  y_lst1=[]
  for i in y1:
      y_lst1.append(i)
  print(' '.join(y_lst1))


  y2=prediction(res[1][0])
  y2=y2.split(' ')
  y_lst2=[]
  for i in y2:
      y_lst2.append(i)
  print(' '.join(y_lst2))

  y3=prediction(res[2][0])
  y3=y3.split(' ')
  y_lst3=[]
  for i in y3:
      y_lst3.append(i)
  print(' '.join(y_lst3))
  print('>'*180)

Input text: 
He's going thru tutorial today? Is today the last lecture?
Actual Output: 
He's going through tutorial today? Is today the last lecture?
Predicted Output for beam==3 : 
He's going thro t tt     t                                 <PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>
He's going thro t tt  a  t                                

In [30]:
import nltk.translate.bleu_score as bleu
bleu_score1=[]
bleu_score2=[]
bleu_score3=[]

for i in range(20):
  b=list(y_test[i:i+1])
  x=model.predict(source_padded_docs_test[i:i+1])

  res=beam_search_decoder(x[0],3)

  y1=prediction(res[0][0])
  y1=y1.split(' ')
  y_lst1=[]
  for i in y1:
    if '<'in i:
      continue
    else:
      y_lst1.append(i)
  bleu_score1.append(bleu.sentence_bleu([b[0].split(),],y_lst1))

  y2=prediction(res[1][0])
  y2=y2.split(' ')
  y_lst2=[]
  for i in y2:
    if i=='<PAD>':
      continue
    else:
      y_lst2.append(i)
  bleu_score2.append(bleu.sentence_bleu([b[0].split(),],y_lst2))

  y3=prediction(res[2][0])
  y3=y3.split(' ')
  y_lst3=[]
  for i in y3:
    if i=='<PAD>':
      continue
    else:
      y_lst3.append(i)
  bleu_score3.append(bleu.sentence_bleu([b[0].split(),],y_lst3))

print("The Average Bleu Score1 is: ",sum(bleu_score1)/20)
print('>'*180)
print("The Average Bleu Score2 is: ",sum(bleu_score2)/20)
print('>'*180)
print("The Average Bleu Score3 is: ",sum(bleu_score3)/20)
print('>'*180)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


The Average Bleu Score1 is:  0.2746536825395667
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
The Average Bleu Score2 is:  0.2718117637320006
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
The Average Bleu Score3 is:  0.273708969340681
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


Model2:

In [35]:
input=tf.keras.layers.Input(shape=(199,))
embed=tf.keras.layers.Embedding(source_vocab_size,256, input_length=source_padded_docs_train.shape[1])(input)
lstm1=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(embed)  
dense=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(512,activation='relu'))(lstm1)
drop=tf.keras.layers.Dropout(0.5)(dense)
output=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(target_vocab_size, activation='softmax'))(drop)
model=tf.keras.models.Model(inputs=input,outputs=output)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 199)]             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 199, 256)          26624     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 199, 256)          394240    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 199, 512)          131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 199, 512)          0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 199, 91)           46683     
Total params: 599,131
Trainable params: 599,131
Non-trainable params: 0
_____________________________________________________

In [36]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [37]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=100,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/100
2/2 [==============================] - 6s 2s/step - loss: 3.5634 - accuracy: 0.3871 - val_loss: 5.6043 - val_accuracy: 0.6420
Epoch 2/100
2/2 [==============================] - 2s 795ms/step - loss: 4.6147 - accuracy: 0.6449 - val_loss: 1.5023 - val_accuracy: 0.6681
Epoch 3/100
2/2 [==============================] - 2s 846ms/step - loss: 1.5370 - accuracy: 0.6425 - val_loss: 1.6905 - val_accuracy: 0.6917
Epoch 4/100
2/2 [==============================] - 2s 814ms/step - loss: 1.7233 - accuracy: 0.6717 - val_loss: 1.5941 - val_accuracy: 0.6683
Epoch 5/100
2/2 [==============================] - 2s 814ms/step - loss: 1.5887 - accuracy: 0.6811 - val_loss: 1.5098 - val_accuracy: 0.6922
Epoch 6/100
2/2 [==============================] - 2s 809ms/step - loss: 1.5349 - accuracy: 0.6805 - val_loss: 1.4723 - val_accuracy: 0.6578
Epoch 7/100
2/2 [==============================] - 2s 815ms/step - loss: 1.4904 - accuracy: 0.6627 - val_loss: 1.4265 - val_accuracy: 0.6714
Epoch 8/100
2/2 

In [38]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=20,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/20
2/2 [==============================] - 2s 824ms/step - loss: 1.1064 - accuracy: 0.7168 - val_loss: 1.1013 - val_accuracy: 0.7168
Epoch 2/20
2/2 [==============================] - 2s 811ms/step - loss: 1.1049 - accuracy: 0.7171 - val_loss: 1.1004 - val_accuracy: 0.7166
Epoch 3/20
2/2 [==============================] - 2s 811ms/step - loss: 1.1036 - accuracy: 0.7175 - val_loss: 1.0921 - val_accuracy: 0.7173
Epoch 4/20
2/2 [==============================] - 2s 810ms/step - loss: 1.1025 - accuracy: 0.7183 - val_loss: 1.0914 - val_accuracy: 0.7173
Epoch 5/20
2/2 [==============================] - 2s 814ms/step - loss: 1.0998 - accuracy: 0.7179 - val_loss: 1.0927 - val_accuracy: 0.7163
Epoch 6/20
2/2 [==============================] - 2s 824ms/step - loss: 1.0994 - accuracy: 0.7181 - val_loss: 1.0879 - val_accuracy: 0.7186
Epoch 7/20
2/2 [==============================] - 2s 811ms/step - loss: 1.0962 - accuracy: 0.7191 - val_loss: 1.0864 - val_accuracy: 0.7168
Epoch 8/20
2/2 [====

In [39]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=50,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/50
2/2 [==============================] - 2s 835ms/step - loss: 1.0831 - accuracy: 0.7205 - val_loss: 1.0790 - val_accuracy: 0.7206
Epoch 2/50
2/2 [==============================] - 2s 829ms/step - loss: 1.0825 - accuracy: 0.7215 - val_loss: 1.0709 - val_accuracy: 0.7206
Epoch 3/50
2/2 [==============================] - 2s 823ms/step - loss: 1.0784 - accuracy: 0.7212 - val_loss: 1.0703 - val_accuracy: 0.7209
Epoch 4/50
2/2 [==============================] - 2s 822ms/step - loss: 1.0772 - accuracy: 0.7223 - val_loss: 1.0614 - val_accuracy: 0.7239
Epoch 5/50
2/2 [==============================] - 2s 819ms/step - loss: 1.0742 - accuracy: 0.7226 - val_loss: 1.0633 - val_accuracy: 0.7229
Epoch 6/50
2/2 [==============================] - 2s 789ms/step - loss: 1.0724 - accuracy: 0.7229 - val_loss: 1.0633 - val_accuracy: 0.7231
Epoch 7/50
2/2 [==============================] - 2s 799ms/step - loss: 1.0703 - accuracy: 0.7237 - val_loss: 1.0605 - val_accuracy: 0.7254
Epoch 8/50
2/2 [====

In [40]:
def prediction(x):

  index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
  index_to_words[0] = '<PAD>'

  y=''.join([index_to_words[prediction] for prediction in x])
  return y
for i in range(20):
  print("Input text: ")
  a=list(X_test[i:i+1])
  print(a[0])

  print("Actual Output: ")
  b=list(y_test[i:i+1])
  print(b[0])

  print("Predicted Output for beam==3 : ")
  x=model.predict(source_padded_docs_test[i:i+1])

  res=beam_search_decoder(x[0],3)

  y1=prediction(res[0][0])
  y1=y1.split(' ')
  y_lst1=[]
  for i in y1:
      y_lst1.append(i)
  print(' '.join(y_lst1))


  y2=prediction(res[1][0])
  y2=y2.split(' ')
  y_lst2=[]
  for i in y2:
      y_lst2.append(i)
  print(' '.join(y_lst2))

  y3=prediction(res[2][0])
  y3=y3.split(' ')
  y_lst3=[]
  for i in y3:
      y_lst3.append(i)
  print(' '.join(y_lst3))
  print('>'*180)

Input text: 
He's going thru tutorial today? Is today the last lecture?
Actual Output: 
He's going through tutorial today? Is today the last lecture?
Predicted Output for beam==3 : 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


He's going thru t tor al    a            the la t lect re<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>
He's going thru t tor al                 the la t lect re<PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>

In [41]:
import nltk.translate.bleu_score as bleu
bleu_score1=[]
bleu_score2=[]
bleu_score3=[]

for i in range(20):
  b=list(y_test[i:i+1])
  x=model.predict(source_padded_docs_test[i:i+1])

  res=beam_search_decoder(x[0],3)

  y1=prediction(res[0][0])
  y1=y1.split(' ')
  y_lst1=[]
  for i in y1:
    if '<'in i:
      continue
    else:
      y_lst1.append(i)
  bleu_score1.append(bleu.sentence_bleu([b[0].split(),],y_lst1))

  y2=prediction(res[1][0])
  y2=y2.split(' ')
  y_lst2=[]
  for i in y2:
    if i=='<PAD>':
      continue
    else:
      y_lst2.append(i)
  bleu_score2.append(bleu.sentence_bleu([b[0].split(),],y_lst2))

  y3=prediction(res[2][0])
  y3=y3.split(' ')
  y_lst3=[]
  for i in y3:
    if i=='<PAD>':
      continue
    else:
      y_lst3.append(i)
  bleu_score3.append(bleu.sentence_bleu([b[0].split(),],y_lst3))

print("The Average Bleu Score1 is: ",sum(bleu_score1)/20)
print('>'*180)
print("The Average Bleu Score2 is: ",sum(bleu_score2)/20)
print('>'*180)
print("The Average Bleu Score3 is: ",sum(bleu_score3)/20)
print('>'*180)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


The Average Bleu Score1 is:  0.2598517965282279
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
The Average Bleu Score2 is:  0.2600204569601833
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
The Average Bleu Score3 is:  0.2662145469430429
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
